# Assignment - D: Build a Regression Model in Keras with Normalized Setting

## Download and Clean Dataset

Let's start by importing the pandas and the Numpy libraries.

In [1]:
import pandas as pd
import numpy as np

We will be using the same dataset provided in the course.

The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:

1. Cement
2. Blast Furnace Slag
3. Fly Ash
4. Water
5. Superplasticizer
6. Coarse Aggregate
7. Fine Aggregate

Let's read the dataset into a *pandas* dataframe.

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Let's do a quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

### Normalize Train Data

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to n_cols since we will need this number when building our network.

In [10]:
n_cols = predictors_norm.shape[1] # number of predictors

## Build a Baseline Model

### Import Keras

The extremely powerful and yet-easy-to-use **Keras** library runs on top of a low-level library such as **TensorFlow** or **PyTorch**. In this case, this assignment will be using **TensorFlow** as a backend.

In [11]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

As you can see, the TensorFlow backend was used to install the Keras library.

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [12]:
from keras.models import Sequential
from keras.layers import Dense

### Build a Neural Network

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model using the following settings:
- 1 hidden layer of *10 nodes*
- *ReLU* activation function
- *adam* optimizer
- *mean squared error* for the loss function

### Generate the Test Dataset

Let's import scikit-learn in order to randomly split the data into a training and test sets

In [14]:
from sklearn.model_selection import train_test_split

Randomly split the data into a training and test sets by holding 30% of the data for testing.

In [15]:
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

### Train and Evaluate the Neural Network Model

Let's call the function now to create our model.

In [16]:
# build the model
model = regression_model()

Next, we will train the model for 50 epochs with 1 verbose setting.

In [17]:
# fit the model - 50 epochs with 1 verbose setting
model.fit(X_train, y_train, epochs=50, verbose=2)

Epoch 1/50
 - 3s - loss: 1588.3237
Epoch 2/50
 - 1s - loss: 1566.2900
Epoch 3/50
 - 1s - loss: 1532.3055
Epoch 4/50
 - 1s - loss: 1480.0267
Epoch 5/50
 - 2s - loss: 1402.4059
Epoch 6/50
 - 1s - loss: 1285.8066
Epoch 7/50
 - 1s - loss: 1120.5236
Epoch 8/50
 - 2s - loss: 917.9469
Epoch 9/50
 - 1s - loss: 711.2770
Epoch 10/50
 - 1s - loss: 543.1236
Epoch 11/50
 - 1s - loss: 415.5637
Epoch 12/50
 - 2s - loss: 319.9016
Epoch 13/50
 - 1s - loss: 252.1567
Epoch 14/50
 - 1s - loss: 208.5978
Epoch 15/50
 - 1s - loss: 185.8726
Epoch 16/50
 - 1s - loss: 175.9153
Epoch 17/50
 - 1s - loss: 168.6962
Epoch 18/50
 - 2s - loss: 164.1804
Epoch 19/50
 - 2s - loss: 160.5983
Epoch 20/50
 - 1s - loss: 156.8517
Epoch 21/50
 - 2s - loss: 153.5599
Epoch 22/50
 - 2s - loss: 150.7646
Epoch 23/50
 - 1s - loss: 147.8270
Epoch 24/50
 - 1s - loss: 145.2490
Epoch 25/50
 - 1s - loss: 142.6707
Epoch 26/50
 - 2s - loss: 140.4321
Epoch 27/50
 - 2s - loss: 138.1624
Epoch 28/50
 - 1s - loss: 135.3561
Epoch 29/50
 - 2s - lo

Next we need to evaluate the model on the test data.

In [18]:
# evaluate the model using the test dataset
loss_val = model.evaluate(X_test, y_test)

# varify the prediction
y_pred = model.predict(X_test)

# output the evaluate value
print(loss_val)

# output the prediction
# print(y_pred)

309/309 [==============================] - 1s 3ms/step
89.78433395435124


Now we need to compute the mean squared error between the predicted concrete strength and the actual concrete strength.

Let's import the *mean_squared_error* function from *scikit-learn*.

In [19]:
from sklearn.metrics import mean_squared_error

Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength.

In [20]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)

print(mean, standard_deviation)

89.78433687867413 0.0


Create a list of 50 mean squared errors and report mean and the standard deviation of the mean squared errors.

In [21]:
total_mean_squared_errors = 50
epochs = 50

mean_squared_errors = []

for i in range(0, total_mean_squared_errors):
    # split the train and test datasets
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    
    # fit the model
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    
    # evaluate the model
    mse = model.evaluate(X_test, y_test, verbose=0)
    print("mean_square_error at " + str(i+1) + " is " + str(mse))
    
    # get the prediction
    y_pred = model.predict(X_test)
    
    # get the mean square error
    mean_square_error = mean_squared_error(y_test, y_pred)
    
    # append result to the mean_squared_errors list
    mean_squared_errors.append(mean_square_error)

# convert to array list
mean_squared_errors = np.array(mean_squared_errors)

# get mean value from the array list
mean = np.mean(mean_squared_errors)

# get standard deviation using the array list
standard_deviation = np.std(mean_squared_errors)

print('\n')
print('\n')
print("The output below is the mean and standard deviation of " + str(total_mean_squared_errors) + " mean squared errors with normalized data.")
print("- Number of epochs for each training: " + str(epochs))
print("- Mean: " + str(mean))
print("- Standard Deviation: " + str(standard_deviation))

mean_square_error at 1 is 48.40187315956288
mean_square_error at 2 is 48.40206325247064
mean_square_error at 3 is 34.63750007160273
mean_square_error at 4 is 35.1231531433303
mean_square_error at 5 is 36.192629422184716
mean_square_error at 6 is 31.592048182070833
mean_square_error at 7 is 33.21035220939365
mean_square_error at 8 is 27.27144757366489
mean_square_error at 9 is 28.11546112109928
mean_square_error at 10 is 24.568263717453842
mean_square_error at 11 is 23.807234001776933
mean_square_error at 12 is 20.92606939544184
mean_square_error at 13 is 22.31453845261756
mean_square_error at 15 is 24.194587411232366
mean_square_error at 16 is 16.360945291117943
mean_square_error at 17 is 20.886876967346783
mean_square_error at 18 is 22.41982281632408
mean_square_error at 19 is 22.269595655422766
mean_square_error at 20 is 23.338118506866753
mean_square_error at 21 is 20.270659684363306
mean_square_error at 22 is 23.06420530547602
mean_square_error at 23 is 18.078450403552996
mean_squa